# Convertir datos y graficar

In [35]:
import MySQLdb
import pandas as pd
from datetime import datetime

db = MySQLdb.connect(host="192.168.0.12",    #  host
                     user="operario",         #  usuario
                     passwd="",  #  password
                     db="dbp8100")        #  nombre de la base de datos

cur = db.cursor()

cur.execute("SELECT * FROM tareaseje")

25175

In [36]:
fecha_actual = datetime.now()
fecha_actual = fecha_actual.strftime("%Y-%m-%d")

In [37]:
# Extraer datos y convertir a df

# Datos mysql
cur.execute('''
            SELECT                 
                tareaseje.NroID,
                tareaseje.Fecha AS Fecha,
                tareaseje.Hora AS Hora,
                formulas.NroID AS IDF,
                formulas.Nombre AS Nombre,
                tareaseje.Set AS Programado,
                (SELECT SUM(dcaptura.Valor) FROM dbp8100.dcaptura WHERE dcaptura.IDT = tareaseje.NroID) AS Dosificado,
                tareaseje.Tiempo AS Tiempo
            FROM 
                dbp8100.tareaseje AS tareaseje
            JOIN 
                dbp8100.formulas AS formulas ON formulas.NroID = tareaseje.IDF
            WHERE
                Fecha >= %s
            GROUP BY
                tareaseje.NroID
            ''', (fecha_actual,))
resultados = cur.fetchall()
# Obtener los nombres de las columnas
columnas = [desc[0] for desc in cur.description]

# Crear un DataFrame de pandas
df = pd.DataFrame(resultados, columns=columnas)

# Cerrar el cursor y la conexión
cur.close()
db.close()

# Mostrar el DataFrame
print(df)

   NroID       Fecha            Hora   IDF                          Nombre  \
0  25170  2025-02-07 0 days 03:16:50  1207      DESARROLLO (LA MALVINA)...   
1  25171  2025-02-07 0 days 04:11:30  1208     TERMINACIÓN (LA MALVINA)...   
2  25172  2025-02-07 0 days 04:55:06  1120      RECRIA 16  AL 15%  A.A....   
3  25173  2025-02-07 0 days 07:55:46  1042           TER MELAZA 12.8%AA...   
4  25174  2025-02-07 0 days 12:24:44   906  FASE 3 STARFEED 15(La Malvina)   
5  25175  2025-02-07 0 days 12:28:50   906  FASE 3 STARFEED 15(La Malvina)   

   Programado  Dosificado          Tiempo  
0      6000.0     6023.80 0 days 00:54:19  
1      6000.0     6023.20 0 days 00:41:14  
2     20000.0    16182.80 0 days 00:00:00  
3     10000.0    10196.95 0 days 01:26:16  
4      2000.0         NaN 0 days 00:03:53  
5      2000.0     2013.10 0 days 00:23:50  


In [38]:
#Corregir los tiempos 0
# Identificar filas con tiempo 0
mask = df['Tiempo'] == pd.Timedelta(0)

# Calcular la diferencia de hora con la siguiente fila
df.loc[mask, 'Tiempo'] = df['Hora'].shift(-1) - df.loc[mask, 'Hora']
df

,NroID,Fecha,Hora,IDF,Nombre,Programado,Dosificado,Tiempo
0,25170,2025-02-07,0 days 03:16:50,1207,DESARROLLO (LA MALVINA)...,6000.0,6023.80,0 days 00:54:19
1,25171,2025-02-07,0 days 04:11:30,1208,TERMINACIÓN (LA MALVINA)...,6000.0,6023.20,0 days 00:41:14
2,25172,2025-02-07,0 days 04:55:06,1120,RECRIA 16 AL 15% A.A....,20000.0,16182.80,0 days 03:00:40
3,25173,2025-02-07,0 days 07:55:46,1042,TER MELAZA 12.8%AA...,10000.0,10196.95,0 days 01:26:16
4,25174,2025-02-07,0 days 12:24:44,906,FASE 3 STARFEED 15(La Malvina),2000.0,NaN,0 days 00:03:53
5,25175,2025-02-07,0 days 12:28:50,906,FASE 3 STARFEED 15(La Malvina),2000.0,2013.10,0 days 00:23:50


In [39]:
df['Rendimiento'] = (df['Dosificado'] / (df['Tiempo'].astype('int64')/3.6e12))/1000
df.dropna(inplace=True)
df

,NroID,Fecha,Hora,IDF,Nombre,Programado,Dosificado,Tiempo,Rendimiento
0,25170,2025-02-07,0 days 03:16:50,1207,DESARROLLO (LA MALVINA)...,6000.0,6023.80,0 days 00:54:19,6.654090
1,25171,2025-02-07,0 days 04:11:30,1208,TERMINACIÓN (LA MALVINA)...,6000.0,6023.20,0 days 00:41:14,8.764559
2,25172,2025-02-07,0 days 04:55:06,1120,RECRIA 16 AL 15% A.A....,20000.0,16182.80,0 days 03:00:40,5.374362
3,25173,2025-02-07,0 days 07:55:46,1042,TER MELAZA 12.8%AA...,10000.0,10196.95,0 days 01:26:16,7.092160
5,25175,2025-02-07,0 days 12:28:50,906,FASE 3 STARFEED 15(La Malvina),2000.0,2013.10,0 days 00:23:50,5.067944


## Gráficos

In [40]:
import plotly.express as px


In [41]:
nombre = (df.index + 1).astype(str) + " - " + df['Nombre']

# Crear gráfico de barras
fig = px.bar(df, x=nombre, y='Dosificado', title='Producción por Producto')

# Mostrar el gráfico
fig.show()

In [42]:
hora = df['Hora'].astype('int64')/3.6e12
# Crear gráfico de barras
fig = px.line(df, x=hora, y='Rendimiento', title='Rendimiento por Producto')

# Mostrar el gráfico
fig.show()

### Etiquetas superiores

In [43]:
#Toneladas elaboradas
tn_total = df['Dosificado'].sum()

print(tn_total)

40439.85


In [44]:
# Horas totales
hs_total = df['Tiempo'].astype('int64').sum() / 3.6e12

print(hs_total)

6.438611111111111


In [45]:
rendimiento_gral = (tn_total / hs_total)/1000
print(rendimiento_gral)

6.280834375943742
